### Importing all the necessary libraries

In [110]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sqlite3
from helper_functions import * #Hope and Adam's
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)
%matplotlib inline

In [2]:
#df = get_clean_df()
#df.info()

In [3]:
#df[['domestic_gross_movie_gross','clean_domestic_gross']]
aekhfbljbv

NameError: name 'aekhfbljbv' is not defined

___
# Reading in the Data

## movie_gross CSV

Domestic Gross and Foreign Gross are hard to read, because numbers are very large, we should consider adjusting the values
probably by making new columns as domestic_gross_mil and foreign_gross_mil

**Null Value Columns:**
* foreign_gross

In [ ]:
movie_gross = pd.read_csv("../Data/bom.movie_gross.csv")
movie_gross.head()

In [ ]:
movie_gross.info()

***
## Internet Movie Database (IMDB)
![alt text](https://raw.githubusercontent.com/learn-co-curriculum/dsc-ai-academy-semester1-capstone/main/movie_data_erd.jpeg "Logo Title Text 1")

In [ ]:
#List of Tables for original table names to read the data in easily
conn = sqlite3.connect("../Data/im.db")
sql_query = """
SELECT name FROM sqlite_master  
WHERE type='table';
"""
pd.read_sql(sql_query,conn)

## Persons table
**Null Value Columns:**
* death_year
* birth_year
* primary_profession

In [ ]:
q = """
SELECT * FROM persons
"""
persons_df = pd.read_sql(q, conn)
persons_df.head()

In [ ]:
persons_df.info()

## Writers table

Many-to-many bridge table with no missing values

In [ ]:
q = """
SELECT * FROM writers
"""
writers_df = pd.read_sql(q, conn)
writers_df.head()

In [ ]:
writers_df.info()

## Directors table

Many-to-many bridge table with no missing values

In [ ]:
q = """
SELECT * FROM directors
"""
directors_df = pd.read_sql(q, conn)
directors_df.head()

In [ ]:
directors_df.info()

## known_for table

Many-to-many bridge table with no missing values

In [ ]:
q = """
SELECT * FROM known_for
"""
known_for_df = pd.read_sql(q, conn)
known_for_df.head()

In [ ]:
known_for_df.info()

## Principals table

**Null Value Columns:**
* job
* characters

Actors don't have non-null values under jobs

Some actors have null values under the character column

In [ ]:
q = """
SELECT * FROM principals
"""
principals_df = pd.read_sql(q, conn)
principals_df.head()

In [ ]:
principals_df.info()

In [ ]:
principals_df[principals_df['category'] == 'actor'].info()

## movie_basics table

**Null Value Columns:**
* runtime_minutes
* genres

In [ ]:
q = """
SELECT * FROM movie_basics
"""
movie_basics_df = pd.read_sql(q, conn)
movie_basics_df.head()

In [ ]:
movie_basics_df.info()

## movie_ratings table
There are no null values, this table should easily join to other tables with movie_id, and the averagerating column seems like it would be a very useful component in measuring the success of a film

In [ ]:
q = """
SELECT * FROM movie_ratings
"""
movie_ratings_df = pd.read_sql(q, conn)
movie_ratings_df.head()

In [ ]:
movie_ratings_df.info()

## movie_akas

**Null Value Columns:**
* region
* language
* types
* attributes

The same movie ID is repeats for versions of movies in different regions, with is_original_title being used to identify which movie is the original.

In [ ]:
q = """
SELECT * FROM movie_akas
"""
movie_akas_df = pd.read_sql(q, conn)
movie_akas_df.head()

In [ ]:
movie_akas_df.info()
#null values under region, language, types, and attributes

***
## rt_movie_info TSV
Contains an id column which relates to the other Rotten Tomatoe dataset (rt_reviews). This is the only dataframe with box_office earnings so it is useful for us to determine success of a movie in theatre, particularly by genre which is also information stored in this dataset.

**Null Value Columns:**
* synopsis
* rating
* genre 
* director
* writer
* theatre_date
* dvd_date
* currency
* box_office
* runtime
* studio  

In [ ]:
rt_movie_info = pd.read_table("../Data/rt.movie_info.tsv", thousands=',')
rt_movie_info.head()

In [ ]:
rt_movie_info.info()

***
## rt_reviews TSV

Again this dataset has an id column which will relate it to the other Rotten Tomatoe dataframe (rt_movie_info)

Rating is given out of 5 stored as string, in order to use that iformation effectively we should have to convert that to a numeric value of sorts. It may also be interesting to just look at the reviews of the top critics only, with some research into what rotten tomatoes defines as a top critic. Regardless we should expect at the very least that top critics would have complete data entries with no null values so sorting for top critic review entries would probably solve a lot of other problems trying to pull information from this dataframe.

**Null Value Columns**
* review
* rating
* critic

In [ ]:
rt_reviews = pd.read_table("../Data/rt.reviews.tsv", encoding = 'latin-1')
rt_reviews
#rating given out of 5 stored as string, might be best to just look at the numerator instead of storing as a fraction
#in a string
#again, id looks like it might connect to other tables hopefully

In [ ]:
rt_reviews.loc[(rt_reviews['top_critic'] == 1)]

In [ ]:
rt_reviews.info()
#null values: review, rating, critic

___
## tmdb movies CSV
TMDB uses a list of genre id's, we need to translate the id's to something comprehendible. Popularity is also a seemingly vague value that a dataframe dictionary would be helpful for. (Note we'll also need to add data dictionaries folder to our repo)

**No Null Value Columns**

In [ ]:
movies = pd.read_csv("../Data/tmdb.movies.csv")
movies.head()
# Need something to translate the genre ids 

In [ ]:
movies.info()

___
## TN movie_budgets CSV

**No Null Value Columns**

In [ ]:
movie_budgets = pd.read_csv("../Data/tn.movie_budgets.csv")
movie_budgets.head()

In [ ]:
movie_budgets.info()

___

## Taking a look at box office numbers by genre

**For reference: Heres a basic look at the rt_movie_info dataframe**

In [ ]:
rt_movie_info.head()

In [ ]:
rt_movie_info.info()

**First we need to make the genre column into a list of strings instead of one big string**

In [ ]:
genres = rt_movie_info['genre']
rt_movie_info['genre'] = genres.str.split(pat="|") #Original string sepates values with | and no spaces
rt_movie_info['genre']

In [ ]:
#rt_movie_info3 = rt_movie_info.explode("genre").reset_index(drop=True)
#rt_movie_info3['release_year'] = rt_movie_info3['theater_date'].dt.year

**Now we pull out each genre and make a new table where:**
* Each row is an individual movie
* Each genre is a column with binary variables
    * 1 = Genre is Present
    * 0 = Genre is not Present

In [ ]:
rt_movie_genres = pd.get_dummies(rt_movie_info['genre'].apply(pd.Series).stack()).sum(level=0)
rt_movie_genres

In [ ]:
rt_movie_genres.info()
#Just checking that that there are no null values or nan columns

**Now we can join back the new rt_movies_genres table back to the original**

In this case we only want to look at box office, theater_date, and dvd_date to get a good idea of what genres are doing well while they're in theatre

In [ ]:
rt_movie_info2 = pd.concat([rt_movie_info[['box_office']], rt_movie_genres], axis = 1)
rt_movie_info2 = pd.concat([rt_movie_info[['theater_date']], rt_movie_info2], axis = 1)
rt_movie_info2 = pd.concat([rt_movie_info[['dvd_date']], rt_movie_info2], axis = 1)

From this data we can also calculate the number of days in theater which we can store as a float to do calculations with

In [ ]:
rt_movie_info2['theater_date'] = pd.to_datetime(rt_movie_info2['theater_date'])
rt_movie_info2['dvd_date'] = pd.to_datetime(rt_movie_info2['dvd_date'])
rt_movie_info2['days_in_theater'] = rt_movie_info2['dvd_date'] - rt_movie_info2['theater_date']
rt_movie_info2['days_in_theater'] = rt_movie_info2['days_in_theater'].dt.days #dt.dats converts timedelta to float

In [ ]:
rt_movie_info2.head()

**Let's create 2 more columns calculating box office earnings/day and box office earnings/week so we have a more reliable way to compare box office earnings between movies that were in theatres for different lengths of time**

In [ ]:
rt_movie_info2['box_office_perday'] = rt_movie_info2['box_office'] / rt_movie_info2['days_in_theater']
rt_movie_info2['box_office_perweek'] = rt_movie_info2['box_office'] / (rt_movie_info2['days_in_theater'] / 7)
rt_movie_info2['release_year'] = rt_movie_info2['theater_date'].dt.year

In [ ]:
#CHANGE BETWEEN PERDAY AND PERWEEK HERE TO GET DIFFERENT OUTPUT 
#YOU WOULD ALSO NEED TO CHANGE GRAPH TITLES AND AXXES LABELS
#PER DAY LOOKS NICER AND DOESN'T MAP THE Y AXIS WITH SCIENTIFIC NOTATION SO THAT'S WHAT I LEAVE AS DEFAULT
rt_movie_info2 = rt_movie_info2.mask(rt_movie_info2 == 1, rt_movie_info2['box_office_perday'], axis = 0)
rt_movie_info2 = rt_movie_info2.fillna(0)
rt_movie_info2

**Now we'll create a list of all the different genres to reference when sorting thedata**

In [ ]:
genres = rt_movie_info2.columns[3:-4] #Use [3:-4] to ignore non-genre columns
genres

**With that list of genres we can create a dictionary of genres following -- {genre:mean earnings per day}**

In [ ]:
genre_dict = {}
for genre in genres: 
    if pd.isna(rt_movie_info2.loc[rt_movie_info2[genre] != 0][genre].mean()) == False:
        genre_dict[genre] = rt_movie_info2.loc[rt_movie_info2[genre] != 0][genre].mean()
genre_dict

**Now we'll graph the findings**

In [ ]:
fig, ax = plt.subplots()
keys = list(genre_dict.keys())
# get values in the same order as keys, and parse percentage values
vals = list(genre_dict.values())
sns.barplot(x=keys, y=vals).set(title='Mean Daily Box Office Earnings by Genre')
ax.set_xlabel('Genre')
ax.set_ylabel("Mean Box Office Earnings per Day")
ax.set_xticklabels(ax.get_xticklabels(),rotation = 80)

**We'll narrow our search to only movies released in theatre 2010 or later**

Then we repeat steps to remake the dictionary and graph again

In [ ]:
today = pd.to_datetime('today').normalize()
rt_movie_info_last_decade = rt_movie_info2.where(today.year - rt_movie_info2['release_year'] <= 20)

In [ ]:
genre_dict = {}
for genre in genres: 
    if pd.isna(rt_movie_info_last_decade.loc[rt_movie_info_last_decade[genre] != 0][genre].mean()) == False:
        genre_dict[genre] = rt_movie_info_last_decade.loc[rt_movie_info_last_decade[genre] != 0][genre].mean()
genre_dict

In [ ]:
fig, ax = plt.subplots()
keys = list(genre_dict.keys())
# get values in the same order as keys, and parse percentage values
vals = list(genre_dict.values())
sns.barplot(x=keys, y=vals).set(title='Mean Daily Box Office Earnings by Genre')
ax.set_xlabel('Genre')
ax.set_ylabel("Mean Box Office Earnings per Day")
ax.set_xticklabels(ax.get_xticklabels(),rotation = 80)

In [ ]:
rt_movie_info_last_decade = rt_movie_info_last_decade[~rt_movie_info_last_decade['release_year'].isna()]

In [ ]:
def normalize_column(values):
    mi = np.min(values) 
    ma = np.max(values) 
    norm = (values - mi)/(ma-mi) 
    return(pd.DataFrame(norm))

In [ ]:
rt_movie_info_last_decade

In [ ]:
AandA = rt_movie_info_last_decade[rt_movie_info_last_decade['Action and Adventure'] != 0]['Action and Adventure'].to_list()
stats.normaltest(AandA)
AandA

In [ ]:
normal_AandA = normalize_column(AandA)
sns.distplot(normal_AandA)

In [ ]:
stats.normaltest(normal_AandA)

## Finding how long different genres are in theater for

In [90]:
rt_movie_info = pd.read_table("../Data/rt.movie_info.tsv", thousands=',')
rt_movie_info3 = rt_movie_info

In [97]:
rt_movie_info3['theater_date'] = pd.to_datetime(rt_movie_info3['theater_date'])
rt_movie_info3['dvd_date'] = pd.to_datetime(rt_movie_info3['dvd_date'])
rt_movie_info3['days_in_theater'] = rt_movie_info3['dvd_date'] - rt_movie_info3['theater_date']
rt_movie_info3['days_in_theater'] = rt_movie_info3['days_in_theater'].dt.days #dt.dats converts timedelta to float
rt_movie_info3['release_year'] = rt_movie_info3['theater_date'].dt.year
today = pd.to_datetime('today').normalize()
rt_movie_info3 = rt_movie_info3[(today.year - rt_movie_info3['release_year'] <= 100)]
rt_movie_info3

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,days_in_theater,release_year
0,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,2013-01-01,$,600000.0,108 minutes,Entertainment One,137,2012
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,2013-01-01,$,600000.0,108 minutes,Entertainment One,137,2012
2,23,A fictional film set in the alluring world of ...,R,Drama,NaN,NaN,2013-12-20,2014-03-18,$,99165609.0,129 minutes,Sony Pictures,88,2013
3,25,"From ancient Japan's most enduring tale, the e...",PG-13,Action and Adventure,Carl Erik Rinsch,Chris Morgan|Hossein Amini,2013-12-25,2014-04-01,$,20518224.0,127 minutes,Universal Pictures,97,2013
4,25,"From ancient Japan's most enduring tale, the e...",PG-13,Drama,Carl Erik Rinsch,Chris Morgan|Hossein Amini,2013-12-25,2014-04-01,$,20518224.0,127 minutes,Universal Pictures,97,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,1975,The year is 1953. A visiting archaeologist cal...,NR,Romance,Vikramaditya Motwane,Vikramaditya Motwane|Bhavani Iyer|Anurag Kashyap,2013-07-03,2015-01-12,$,581813.0,142 minutes,NaN,558,2013
262,1976,"Embrace of the Serpent features the encounter,...",NR,Action and Adventure,Ciro Guerra,Ciro Guerra|Jacques Toulemonde Vidal,2016-02-17,2016-06-21,$,1320005.0,123 minutes,Buffalo Films,125,2016
263,1976,"Embrace of the Serpent features the encounter,...",NR,Art House and International,Ciro Guerra,Ciro Guerra|Jacques Toulemonde Vidal,2016-02-17,2016-06-21,$,1320005.0,123 minutes,Buffalo Films,125,2016
264,1981,"Money, Fame and the Knowledge of English. In I...",NR,Comedy,Gauri Shinde,Gauri Shinde,2012-10-05,2012-11-20,$,1416189.0,129 minutes,Eros Entertainment,46,2012


In [98]:
genres = rt_movie_info3['genre']
rt_movie_info3['genre'] = genres.str.split(pat="|")
rt_movie_info3 = rt_movie_info3.explode("genre").reset_index(drop=True)
rt_movie_info3

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio,days_in_theater,release_year
0,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,2013-01-01,$,600000.0,108 minutes,Entertainment One,137,2012
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,2012-08-17,2013-01-01,$,600000.0,108 minutes,Entertainment One,137,2012
2,23,A fictional film set in the alluring world of ...,R,Drama,NaN,NaN,2013-12-20,2014-03-18,$,99165609.0,129 minutes,Sony Pictures,88,2013
3,25,"From ancient Japan's most enduring tale, the e...",PG-13,Action and Adventure,Carl Erik Rinsch,Chris Morgan|Hossein Amini,2013-12-25,2014-04-01,$,20518224.0,127 minutes,Universal Pictures,97,2013
4,25,"From ancient Japan's most enduring tale, the e...",PG-13,Drama,Carl Erik Rinsch,Chris Morgan|Hossein Amini,2013-12-25,2014-04-01,$,20518224.0,127 minutes,Universal Pictures,97,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,1975,The year is 1953. A visiting archaeologist cal...,NR,Romance,Vikramaditya Motwane,Vikramaditya Motwane|Bhavani Iyer|Anurag Kashyap,2013-07-03,2015-01-12,$,581813.0,142 minutes,NaN,558,2013
262,1976,"Embrace of the Serpent features the encounter,...",NR,Action and Adventure,Ciro Guerra,Ciro Guerra|Jacques Toulemonde Vidal,2016-02-17,2016-06-21,$,1320005.0,123 minutes,Buffalo Films,125,2016
263,1976,"Embrace of the Serpent features the encounter,...",NR,Art House and International,Ciro Guerra,Ciro Guerra|Jacques Toulemonde Vidal,2016-02-17,2016-06-21,$,1320005.0,123 minutes,Buffalo Films,125,2016
264,1981,"Money, Fame and the Knowledge of English. In I...",NR,Comedy,Gauri Shinde,Gauri Shinde,2012-10-05,2012-11-20,$,1416189.0,129 minutes,Eros Entertainment,46,2012


In [99]:
rt_movie_info4 = rt_movie_info3.dropna(subset=['dvd_date'])

In [100]:
genre_in_theater

,days_in_theater
genre,
Documentary,164.0
Faith and Spirituality,139.0
Special Interest,137.0
Art House and International,116.0
Musical and Performing Arts,113.5
Horror,109.0
Cult Movies,109.0
Science Fiction and Fantasy,102.0
Mystery and Suspense,102.0


In [101]:
genre_in_theater.loc[["Horror"]]['days_in_theater']

genre
Horror    109.0
Name: days_in_theater, dtype: float64

In [102]:
genre_in_theater = rt_movie_info4.groupby("genre").median()[["days_in_theater"]].sort_values("days_in_theater", ascending=False)
#genre_in_theater[genre_in_theater['genre'] == 'Horror']
genre_in_theater


,days_in_theater
genre,
Documentary,164.0
Faith and Spirituality,139.0
Special Interest,137.0
Art House and International,116.0
Musical and Performing Arts,113.5
Horror,109.0
Cult Movies,109.0
Science Fiction and Fantasy,102.0
Mystery and Suspense,102.0


In [64]:
rt_movie_info3[rt_movie_info3['genre'] == 'Science Fiction and Fantasy']

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
4,3,"New York City, not-too-distant-future: Eric Pa...",R,Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,600000.0,108 minutes,Entertainment One
18,15,When a diamond expedition in the Congo is lost...,PG-13,Science Fiction and Fantasy,Frank Marshall,John Patrick Shanley,"Jun 9, 1995","Jul 27, 1999",NaN,NaN,108 minutes,NaN
36,25,"From ancient Japan's most enduring tale, the e...",PG-13,Science Fiction and Fantasy,Carl Erik Rinsch,Chris Morgan|Hossein Amini,"Dec 25, 2013","Apr 1, 2014",$,20518224.0,127 minutes,Universal Pictures
48,29,This futuristic adventure stars Lee Majors as ...,PG,Science Fiction and Fantasy,Martyn Burke,NaN,"Apr 1, 1981","May 10, 2011",NaN,NaN,101 minutes,NaN
91,55,A progressive scientist builds a machine that ...,NR,Science Fiction and Fantasy,Henning Schellerup,NaN,NaN,NaN,NaN,NaN,99 minutes,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3470,1944,"In the 3D action-comedy The Green Hornet, Brit...",PG-13,Science Fiction and Fantasy,Michel Gondry,Evan Goldberg|Seth Rogen,"Jan 14, 2011","May 3, 2011",$,98000000.0,119 minutes,Sony Pictures
3517,1972,Stephen Somers wrote and directed this,R,Science Fiction and Fantasy,Stephen Sommers,Stephen Sommers,"Jan 30, 1998","Oct 13, 1998",NaN,NaN,106 minutes,NaN
3521,1973,Agency tackles the question of the efficiency ...,R,Science Fiction and Fantasy,George Kaczender,Noel Hynd|Paul Gottlieb,"Jan 1, 2004","Aug 2, 2005",NaN,NaN,94 minutes,NaN
3536,1980,A band of renegades on the run in outer space ...,PG-13,Science Fiction and Fantasy,Joss Whedon,Joss Whedon,"Sep 30, 2005","Dec 20, 2005",$,25335935.0,119 minutes,Universal Pictures


### We'll put this all in a function that can be called to return the median number of days a specific genre has been in theaters

In [103]:
def median_days_in_theater(genre):
    rt_movie_info = pd.read_table("../Data/rt.movie_info.tsv", thousands=',')
    rt_movie_info3 = rt_movie_info
    
    rt_movie_info3['theater_date'] = pd.to_datetime(rt_movie_info3['theater_date'])
    rt_movie_info3['dvd_date'] = pd.to_datetime(rt_movie_info3['dvd_date'])
    rt_movie_info3['days_in_theater'] = rt_movie_info3['dvd_date'] - rt_movie_info3['theater_date']
    rt_movie_info3['days_in_theater'] = rt_movie_info3['days_in_theater'].dt.days #dt.dats converts timedelta to float
    rt_movie_info3['release_year'] = rt_movie_info3['theater_date'].dt.year
    today = pd.to_datetime('today').normalize()
    rt_movie_info3 = rt_movie_info3[(today.year - rt_movie_info3['release_year'] <= 10)]
    
    genres = rt_movie_info3['genre']
    rt_movie_info3['genre'] = genres.str.split(pat="|")
    rt_movie_info3 = rt_movie_info3.explode("genre").reset_index(drop=True)
    
    genre_in_theatre = rt_movie_info4.groupby("genre").median()[["days_in_theater"]].sort_values("days_in_theater", ascending=False)
    
    median_days = genre_in_theater.loc[[genre]]['days_in_theater']
    median_days = median_days[0]
    
    return median_days

### And for a list of genre's for the function above. here's another function you can call

In [104]:
def genre_list():
    rt_movie_info = pd.read_table("../Data/rt.movie_info.tsv", thousands=',')
    rt_movie_info3 = rt_movie_info

    genres = rt_movie_info3['genre']
    rt_movie_info3['genre'] = genres.str.split(pat="|")
    rt_movie_info3 = rt_movie_info3.explode("genre").reset_index(drop=True)
    genres = rt_movie_info3['genre']
    
    return genres.unique()

In [105]:
genre_list()

array(['Action and Adventure', 'Classics', 'Drama',
       'Science Fiction and Fantasy', 'Musical and Performing Arts',
       'Mystery and Suspense', 'Romance', 'Kids and Family', 'Comedy',
       nan, 'Documentary', 'Special Interest',
       'Art House and International', 'Horror', 'Western', 'Television',
       'Sports and Fitness', 'Animation', 'Faith and Spirituality',
       'Cult Movies', 'Anime and Manga', 'Gay and Lesbian'], dtype=object)

In [106]:
median_days_in_theater('Action and Adventure')

<ipython-input-103-7165fffb2f3f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rt_movie_info3['genre'] = genres.str.split(pat="|")


97.0

In [107]:
AaA = median_days_in_theater('Action and Adventure')

<ipython-input-103-7165fffb2f3f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rt_movie_info3['genre'] = genres.str.split(pat="|")


95.0 102.0


<ipython-input-103-7165fffb2f3f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rt_movie_info3['genre'] = genres.str.split(pat="|")
<ipython-input-103-7165fffb2f3f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rt_movie_info3['genre'] = genres.str.split(pat="|")
